In [1]:
sentence = "I'd like to see a happy movie"

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('final_english_dataset_with_preprocess_on_overview.csv')

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [5]:
from nltk.corpus import wordnet as wn

# Function to get synonyms and related words for a given word
def get_related_words(word):
    related_words = set()
    for synset in wn.synsets(word):
        for hypernym in synset.hypernyms():
            for lemma in hypernym.lemmas():
                related_words.add(lemmatizer.lemmatize(lemma.name().lower()))
    return related_words


In [6]:
def get_synonyms(word):
    synonyms = set()
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemmatizer.lemmatize(lemma.name().lower()))
    return synonyms

In [7]:
def get_derived_words(word):
    derived_words = set()
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            forms = lemma.derivationally_related_forms()
            for form in forms:
                name = form.name()
                derived_words.add(lemmatizer.lemmatize(name.lower()))    
    return derived_words

In [8]:
def expand_genre_keywords(genres):
    expanded_keywords = {}
    for genre in genres:
        keywords = set()  
        keywords.add(genre)
        if genre == "Drama":
            keywords.add("sad")
        if genre == "Comedy": 
            keywords.add("happy")
        for keyword in keywords.copy():
            synonyms = get_synonyms(keyword)
            related_words = get_related_words(keyword)
            derived_words = get_derived_words(keyword)
            keywords.update(synonyms)
            keywords.update(related_words)
            keywords.update(derived_words)
        if genre in keywords:
            keywords.remove(genre)
        expanded_keywords[genre] = list(keywords)  
    return expanded_keywords


In [9]:
def get_list_data (column_name, data) :
    liste_data = set()
    for genres_list in data[column_name]:
        genres_list = eval(genres_list)  
        liste_data.update(genres_list)
    return liste_data

In [10]:
all_genres = get_list_data('genres', data)
genre_keywords = expand_genre_keywords(all_genres)

In [11]:
import joblib
joblib.dump(genre_keywords, 'expanded_keywords.pkl')


['expanded_keywords.pkl']

In [12]:
import re
def find_genre(sentence, expanded_keywords):
    words = [lemmatizer.lemmatize(word.lower()) for word in re.split(r'\W+', sentence) if word]
    words = [word for word in words if word not in ['film', 'movie']]
    for genre, keywords in expanded_keywords.items():
        if any(word in words for word in keywords):
            return genre
    return None

In [13]:
find_genre(sentence, genre_keywords)

'Comedy'